In [1]:
import pandas as pd

from config.development import DATABASE

In [3]:
df = pd.read_sql('articles', DATABASE, index_col = 'id')

In [4]:
df.sample(10)

,url,title,date,body,author
id,,,,,
7176,https://www.inform.kz/ru/pravitel-stvennaya-ne...,ПРАВИТЕЛЬСТВЕННАЯ НЕДЕЛЯ: В Казахстане начинае...,2012-03-16,АСТАНА. 16 марта. КАЗИНФОРМ - В Казахстане нач...,Мейрамбек Байгарин
1467,https://www.inform.kz/ru/glava-gosudarstva-pod...,Глава государства подписал Закон РК «О телерад...,2012-01-18,АСТАНА. 18 января. КАЗИНФОРМ - Глава государст...,None
8512,https://www.inform.kz/ru/srochno-soversheno-na...,СРОЧНО: Совершено нападение на пост транспортн...,2012-04-02,"АСТАНА. 2 апреля. КАЗИНФОРМ - Сегодня ночью, в...",None
21383,https://www.inform.kz/ru/mangystauskoe-gkp-rab...,"Мангыстауское ГКП, работающее в коммунальной с...",2012-08-13,"АКТАУ. 13 августа. КАЗИНФОРМ - ГКП «ТВСиВ», от...",Александра Данилова
6715,https://www.inform.kz/ru/v-kazahstane-razrabat...,В Казахстане разрабатывается Программа развити...,2012-03-12,АСТАНА. 12 марта. КАЗИНФОРМ - Минэкономразвити...,None
19109,https://www.inform.kz/ru/lyudi-perestali-pryat...,Люди перестали прятать больной скот и активно ...,2012-07-19,"АСТАНА. 19 июля. КАЗИНФОРМ - Люди перестали, к...",None
2765,https://www.inform.kz/ru/ogranicheno-dvizhenie...,Ограничено движение в 4 областях Казахстана -...,2012-01-31,АСТАНА. 31 января. КАЗИНФОРМ - По состоянию на...,None
26346,https://www.inform.kz/ru/moshennika-obmanyvavs...,"Мошенника, обманывавшего сельчан при помощи ба...",2012-10-04,ШЫМКЕНТ. 4 октября. КАЗИНФОРМ - Молодого челов...,None
18581,https://www.inform.kz/ru/komanda-astana-finish...,Команда «Астана» финишировала на Шелковом пути...,2012-07-13,АЛМАТЫ. 13 июля. КАЗИНФОРМ - В Майкопе наканун...,Мустафина Сара


# Text preprocessing
## Basic preprocessing
* Lower-case
* Remove numbers and non-letter characters
* Lemmatization

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import re

from tqdm.notebook import tqdm
tqdm.pandas()

from pymystem3 import Mystem
mystem = Mystem()


def preprocess(text):
    global mystem
    text = text.lower()
    text = re.sub('[\W_\d]+', ' ', text)
    text = mystem.lemmatize(text)
    text = ''.join(text)
    return text            

df['body_preprocessed'] = df.body.progress_apply(preprocess)
df['title_preprocessed'] = df.title.progress_apply(preprocess)

## Search using TF-IDF

In [ ]:
tfidf_body = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.9,
    min_df = 3,
    max_features = 10000,
)

tfidf_title = TfidfVectorizer(
    ngram_range = (1,2),
    max_df = 0.9,
    min_df = 3,
    max_features = 5000,
)

tfidf_body_matrix = tfidf_body.fit_transform(df.body_preprocessed)
tfidf_title_matrix = tfidf_title.fit_transform(df.title_preprocessed)

### Get top __n__ words the article

In [64]:
n, top_n = 1, 5
a_ind = df.sample(n).index
articles = df.body_preprocessed[a_ind]
def top_n_tokens(tfidf_body: TfidfVectorizer, articles: pd.Series, top_n: int) -> list:
    a_tfidf = tfidf_body.transform(articles).todense().A
    a_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :n]
    m = tfidf_body.get_feature_names()
    
    a_tfidf = tfidf_body.transform(articles).todense().A
    a_ind_srt = np.flip(np.argsort(a_tfidf, axis=-1), axis=-1)[:, :top_n]
    fn = tfidf_body.get_feature_names()

    return [(indT, [(fn[i], a_tfidf[ind, i]) for i in a_ind_srt[ind]]) for ind, indT in enumerate(articles.index)]

top_n_tokens(tfidf_body, articles, top_n)

[(19089,
  [('tod', 0.20681984481075893),
   ('сессия', 0.18064729916815608),
   ('тенге', 0.17473248636159405),
   ('расчет', 0.1734085934335236),
   ('закрытие сессия', 0.16403944705096613)])]

In [70]:
from sklearn.metrics.pairwise import linear_kernel

q = "Если хочешь выиграть меня, то сука"

def get_top_n(query, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix, n=10):
    q_transformed = preprocess(q)
    q_transformed_body = tfidf_body.transform([q_transformed])
    q_transformed_title = tfidf_title.transform([q_transformed])

    dist_body = linear_kernel(q_transformed_body, tfidf_body_matrix).flatten()
    dist_title = linear_kernel(q_transformed_title, tfidf_title_matrix).flatten()

    b_weight = 0.3
    t_weight = 1 - b_weight

    dist_weighted = b_weight * dist_body + t_weight * dist_title
    top_n = dist_weighted.argsort()[-n:][::-1]
    return list(zip(top_n, dist_weighted[top_n]))

get_top_n(q, tfidf_body, tfidf_body_matrix, tfidf_title, tfidf_title_matrix)

[(17617, 0.3569253429608193),
 (17527, 0.3365570339254973),
 (10765, 0.29488389571984813),
 (29970, 0.27606608574326563),
 (6696, 0.25999235343192195),
 (27363, 0.25891006645194387),
 (12324, 0.25526198778771575),
 (9353, 0.2363623923202727),
 (29377, 0.23379512708371142),
 (19861, 0.23110442680149904)]

In [71]:
import pickle

In [72]:
SEARCH_FOLDER = './app/models/search/'

In [73]:
pickle.dump(tfidf_body, open(SEARCH_FOLDER + 'tfidf_body.pkl','wb'))
pickle.dump(tfidf_body_matrix, open(SEARCH_FOLDER + 'tfidf_body_matrix.pkl','wb'))
pickle.dump(tfidf_title, open(SEARCH_FOLDER + 'tfidf_title.pkl','wb'))
pickle.dump(tfidf_title_matrix, open(SEARCH_FOLDER + 'tfidf_title_matrix.pkl','wb'))